# 실습 4: Agent & Memory 생성

## 목표
- config.py 생성 (Runtime 설정)
- Memory Manager & Hooks 구현
- Agent Factory 생성

---
## 📚 AgentCore Memory 활용

### Memory Hook 동작 흐름

```
1. Agent 초기화 → on_agent_initialized()
   └── Short-term: 최근 대화 로드 (get_last_k_turns)
   
2. 메시지 추가 → on_message_added()
   └── Short-term: 대화 저장 (add_turns)
   └── Long-term: 관련 기억 검색 (retrieve_memories)
   
3. 응답 완료 → on_after_invocation()
   └── Long-term: 대화 저장 (create_event)
```

### Hook 등록
- `register_hooks(registry)` 메서드로 이벤트 콜백 등록
- `AgentInitializedEvent`, `MessageAddedEvent`, `AfterInvocationEvent`

---
## 4-1. 설정 확인

In [ ]:
from workshop_config import REGION, MEMORY_ID, MODEL_ID, WINDY_API_KEY, BROWSER_MODEL_ID

print(f"🌍 Region: {REGION}")
print(f"🧠 Memory ID: {MEMORY_ID}")
print(f"🤖 Model: {MODEL_ID}")

if not MEMORY_ID:
    print("\n⚠️ MEMORY_ID가 없습니다. lab2를 먼저 실행하세요.")

## 4-2. 디렉토리 구조 생성

In [ ]:
import os

dirs = [
    '../agent/deploy/runtime/agent',
    '../agent/deploy/runtime/memory',
    '../agent/deploy/runtime/tool_use',
]

for d in dirs:
    os.makedirs(d, exist_ok=True)
    init_file = os.path.join(d, '__init__.py')
    if not os.path.exists(init_file):
        open(init_file, 'w').close()

print("✅ 디렉토리 구조 생성 완료")

---
## 4-3. config.py 생성 (AgentCore Runtime용)

Runtime에서 사용할 설정 파일입니다.

In [ ]:
from workshop_config import REGION, MEMORY_ID, MODEL_ID, WINDY_API_KEY, BROWSER_MODEL_ID

config_content = f'''"""Configuration settings for Bedrock AgentCore"""

# Model Configuration
MODEL_ID = "{MODEL_ID}"

# AWS Configuration
REGION = "{REGION}"

# Memory Configuration
MEMORY_ID = "{MEMORY_ID}"

# Windy API Configuration
WINDY_API_KEY = "{WINDY_API_KEY}"
WINDY_API_URL = "https://api.windy.com/api/point-forecast/v2"

# Agent Configuration
AGENT_NAME = "FireCommandAssistant"
AGENT_SYSTEM_PROMPT = """

당신은 AI 어시스턴트입니다.
두 가지의 역할을 수행합니다. 
첫 번째 일반적인 대화에 대하여 어시스턴트 역할을 합니다.  
두 번째 위도/경도 등 정보를 전달 받아 화재가 발생했다는 가정의 상황이 발생하면, 소방서의 대응을 지원하는 화재 신고 접수 시, 사용 가능한 도구들을 판단/활용 하여 소방대 출동에 필요한 정보를 수집하고 정리합니다.

## 필수 규칙
1. 모든 응답은 한글로 작성한다.
2. **도구를 사용하기 전에 반드시 "~를 확인하기 위해 [도구명]을 사용합니다"라고 먼저 말한다.**
3. **도구 실행 후 반드시 결과를 요약하여 사용자에게 설명한다.**
4. 도구만 호출하고 끝내지 않는다. 반드시 최종 답변을 제공한다.

## 도구별 출력 형식

### 소방서 검색 (find_fire_station)
- 사용 전: "가까운 소방서를 찾기 위해 find_fire_station을 사용합니다."
- 결과 출력: <event type="geocode">위도,경도,소방서이름</event>

### 기상 정보 (get_weather_info)
- 사용 전: "현장 기상 상황을 확인하기 위해 get_weather_info를 사용합니다."
- 결과 출력: <event type="windy">위도,경도,온도,풍속,풍향,습도,기압,주소</event>
 
### cctv (find_cctv)
- 결과 출력: <event type="cctv">위도,경도,설치주소,스트리밍url</event>

## 응답 흐름
1. 도구 사용 목적 설명
2. 도구 실행
3. **도구 결과 원본 출력**
4. 결과 해석 및 요약
5. 최종 종합 답변
"""

# Browser Tool Configuration
BROWSER_MODEL_ID = "{BROWSER_MODEL_ID}"
BROWSER_SYSTEM_PROMPT = "You are a news intelligence agent."
'''

with open('../agent/deploy/runtime/config.py', 'w') as f:
    f.write(config_content)

print("✅ config.py 생성 완료")

---
## 4-4. Memory Manager 생성

In [ ]:
%%writefile ../agent/deploy/runtime/memory/manager.py
"""Memory session management"""

from bedrock_agentcore.memory.session import MemorySession, MemorySessionManager
from config import MEMORY_ID, REGION

_session_manager = None


def get_memory_session_manager() -> MemorySessionManager:
    """Get or create memory session manager"""
    global _session_manager
    if _session_manager is None:
        _session_manager = MemorySessionManager(memory_id=MEMORY_ID, region_name=REGION)
        print(f"✅ Session manager initialized for memory: {MEMORY_ID}")
    return _session_manager


def create_memory_session(actorId: str, sessionId: str) -> MemorySession:
    """Create a memory session for the given actor and session"""
    manager = get_memory_session_manager()
    session = manager.create_memory_session(
        actor_id=actorId,
        session_id=sessionId
    )
    print(f"🔧 Memory session created (actor: {actorId}, session: {sessionId})")
    return session

---
## 4-5. Memory Hooks 생성

In [ ]:
%%writefile ../agent/deploy/runtime/memory/hooks.py
"""Memory hooks for Agent lifecycle events"""

from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import ConversationalMessage, MessageRole
from bedrock_agentcore.memory.session import MemorySessionManager
from strands.hooks import AgentInitializedEvent, AfterInvocationEvent, MessageAddedEvent, HookProvider, HookRegistry

from config import MEMORY_ID, REGION


class ShortTermMemoryHookProvider(HookProvider):
    def __init__(self, memory_session: MemorySessionManager):
        self.memory_session = memory_session

    def on_agent_initialized(self, event: AgentInitializedEvent):
        """에이전트 시작 시 최근 대화 기록을 로드합니다"""
        try:
            recent_turns = self.memory_session.get_last_k_turns(k=5)

            if recent_turns:
                context_messages = []
                for turn in recent_turns:
                    for message in turn:
                        if hasattr(message, 'role') and hasattr(message, 'content'):
                            role = message['role']
                            content = message['content']
                        else:
                            role = message.get('role', 'unknown')
                            content = message.get('content', {}).get('text', '')
                        
                        content_str = str(content)
                        context_messages.append(f"{role}: {content_str}")
                        print(f"📖 {role}: {content_str[:100]}...")

                context = "\n".join(context_messages)
                event.agent.system_prompt += f"\n\nRecent conversation:\n{context}"
                print(f"✅ {len(recent_turns)}개 대화 로드 완료")

        except Exception as e:
            print(f"❌ 메모리 로드 오류: {e}")

    def on_message_added(self, event: MessageAddedEvent):
        """메시지를 메모리에 저장합니다"""
        messages = event.agent.messages
        try:
            if messages and len(messages) > 0 and messages[-1]["content"][0].get("text"):
                message_text = messages[-1]["content"][0]["text"]
                message_role = MessageRole.USER if messages[-1]["role"] == "user" else MessageRole.ASSISTANT

                print(f"💾 {message_role.value}: {message_text[:100]}...")

                result = self.memory_session.add_turns(
                    messages=[ConversationalMessage(message_text, message_role)]
                )

                event_id = result['eventId']
                print(f"✅ 메시지 저장 완료 (Event ID: {event_id})")

        except Exception as e:
            print(f"❌ 메모리 저장 오류: {e}")

    def register_hooks(self, registry: HookRegistry):
        registry.add_callback(MessageAddedEvent, self.on_message_added)
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)
        print("✅ 메모리 훅 등록 완료")


class LongTermMemoryHookProvider(HookProvider):
    """장기 메모리 관리 훅"""

    def __init__(self, memory_id: str, client: MemoryClient):
        self.memory_id = memory_id
        self.client = client

    def retrieve_memories(self, event: MessageAddedEvent):
        """사용자 메시지 처리 전 관련 메모리를 검색합니다"""
        messages = event.agent.messages
        if messages[-1]["role"] == "user" and "toolResult" not in messages[-1]["content"][0]:
            user_message = messages[-1]["content"][0].get("text", "")

            try:
                actor_id = event.agent.state.get("actor_id")
                if not actor_id:
                    print("❌ actor_id 없음")
                    return

                namespace = f"/actors/{actor_id}"
                print(f"🔍 메모리 검색: {user_message[:100]}...")

                memories = self.client.retrieve_memories(
                    memory_id=self.memory_id,
                    namespace=namespace,
                    query=user_message
                )

                memory_context = []
                for memory in memories:
                    if isinstance(memory, dict):
                        content = memory.get('content', {})
                        if isinstance(content, dict):
                            text = content.get('text', '').strip()
                            if text:
                                memory_context.append(text)
                                print(f"📖 {text[:100]}...")

                if memory_context:
                    context_text = "\n".join(memory_context)
                    original_text = messages[-1]["content"][0].get("text", "")
                    messages[-1]["content"][0]["text"] = (
                        f"{original_text}\n\nPrevious context: {context_text}"
                    )
                    print(f"✅ {len(memory_context)}개 메모리 검색 완료")
                else:
                    print("ℹ️ 검색된 메모리 없음")

            except Exception as e:
                print(f"❌ 메모리 검색 실패: {e}")

    def save_memories(self, event: AfterInvocationEvent):
        """에이전트 응답 후 대화를 저장합니다"""
        try:
            messages = event.agent.messages
            if len(messages) >= 2 and messages[-1]["role"] == "assistant":
                user_msg = None
                assistant_msg = None

                for msg in reversed(messages):
                    if msg["role"] == "assistant" and not assistant_msg:
                        assistant_msg = msg["content"][0]["text"]
                    elif msg["role"] == "user" and not user_msg and "toolResult" not in msg["content"][0]:
                        user_msg = msg["content"][0]["text"]
                        break

                if user_msg and assistant_msg:
                    actor_id = event.agent.state.get("actor_id")
                    session_id = event.agent.state.get("session_id")

                    if not actor_id or not session_id:
                        print("❌ actor_id 또는 session_id 없음")
                        return

                    print(f"💾 장기 메모리 저장: {user_msg[:50]}...")

                    result = self.client.create_event(
                        memory_id=self.memory_id,
                        actor_id=actor_id,
                        session_id=session_id,
                        messages=[(user_msg, "USER"), (assistant_msg, "ASSISTANT")]
                    )
                    print(f"✅ 장기 메모리 저장 완료")

        except Exception as e:
            print(f"❌ 장기 메모리 저장 실패: {e}")

    def register_hooks(self, registry: HookRegistry) -> None:
        registry.add_callback(MessageAddedEvent, self.retrieve_memories)
        registry.add_callback(AfterInvocationEvent, self.save_memories)
        print("✅ 장기 메모리 훅 등록 완료")

---
## 4-6. Agent Factory 생성

In [ ]:
%%writefile ../agent/deploy/runtime/agent/factory.py
"""Agent factory for creating configured agents"""

from strands import Agent
from strands.models import BedrockModel

from bedrock_agentcore.memory import MemoryClient
from config import MODEL_ID, AGENT_NAME, AGENT_SYSTEM_PROMPT, MEMORY_ID, REGION
from memory.hooks import ShortTermMemoryHookProvider, LongTermMemoryHookProvider
from memory.manager import create_memory_session
from tool_use.disaster_tools import wikipedia, find_fire_station, get_weather_info, find_cctv


def create_agent(actorId: str, sessionId: str) -> Agent:
    """메모리와 도구가 설정된 에이전트를 생성합니다"""
    userSession = create_memory_session(actorId, sessionId)
    memory_client = MemoryClient(region_name=REGION)
    model = BedrockModel(model_id=MODEL_ID)
    
    agent = Agent(
        name=AGENT_NAME,
        model=model,
        system_prompt=AGENT_SYSTEM_PROMPT,
        hooks=[
            ShortTermMemoryHookProvider(userSession),
            LongTermMemoryHookProvider(MEMORY_ID, memory_client)
        ],
        tools=[wikipedia, find_fire_station, find_cctv, get_weather_info],
        state={"actor_id": actorId, "session_id": sessionId}
    )
    
    print(f"✅ 에이전트 생성 완료 (actor: {actorId}, session: {sessionId})")
    return agent

---
## ✅ 실습 4 완료!

### 생성된 파일
- `agent/deploy/runtime/config.py`
- `agent/deploy/runtime/memory/manager.py`
- `agent/deploy/runtime/memory/hooks.py`
- `agent/deploy/runtime/agent/factory.py`

👉 다음: **lab5_create_tools.ipynb**